In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np


In [3]:
from dspy.api import get_api
from dspy.api.bybit import Config

# <font color="grey">Simple ByBit API</font>

In [4]:
# Fixed parameters
API_KEY = "StMMBpVlRHbiZF5tki"
API_SECRET = "DOe9UXJgeWLpTKWBibHBpjIDW0UpSON9FuUp"
SYMBOL = 'BTCUSDT'
COIN = 'USDT'

# Parameters to be optimized
EQUITY = 2.0
RANGE = 0.005
NUM_ORDERS = 5
POLLING_RATE = 1
TP_DIST = 0.0005
STOP_DIST = 0 #0.025

config = Config(API_KEY, 
                API_SECRET, 
                SYMBOL, 
                COIN, 
                EQUITY, 
                RANGE, 
                NUM_ORDERS, 
                POLLING_RATE, 
                TP_DIST, 
                STOP_DIST)

In [12]:
api = get_api('bybit', config=config)

In [13]:
symbols = ['BTCUSDT', 'ETHUSDT']

###  <a id='chapter1'> <font color="grey">Prices and Volumes</font></a>

In [14]:
mid = api.get_mid(symbols[1])
print(f"Last price: {mid}")

Last price:  2220.98


In [18]:
ask = api.get_ask(symbols[0])
bid = api.get_bid(symbols[0])
print(f"Bid: {bid}")
print(f"Ask: {ask}")

Bid:  [85871.8, 3.761]
Ask:  [85871.9, 6.073]


In [20]:
orderbook = api.get_orderbook(symbols[0], depth=5)
print(f"Bids: {orderbook['b']}")
print(f"Asks: {orderbook['a']}")

Bids: [[8.58546e+04 8.70400e+00]
 [8.58545e+04 3.49000e-01]
 [8.58540e+04 7.20000e-02]
 [8.58536e+04 4.90000e-02]
 [8.58535e+04 4.30000e-02]]
Asks: [[8.58547e+04 4.80000e-02]
 [8.58549e+04 1.00000e+01]
 [8.58560e+04 2.00000e-03]
 [8.58569e+04 2.00000e-03]
 [8.58578e+04 2.00000e-03]]


In [22]:
print(f"Latency: {api.get_latency(symbols[1])}")

Latency: 2


In [31]:
trades = api.get_trades(symbols[0], limit=5)
trades

[{'ts': '1740908585370', 'price': 85961.3, 'qty': 0.001, 'side': -1},
 {'ts': '1740908585360', 'price': 85961.4, 'qty': 0.01, 'side': 1},
 {'ts': '1740908584842', 'price': 85961.4, 'qty': 0.001, 'side': 1},
 {'ts': '1740908584454', 'price': 85961.4, 'qty': 0.01, 'side': 1},
 {'ts': '1740908583997', 'price': 85961.4, 'qty': 0.008, 'side': 1}]

### Account info

In [32]:
balance = api.get_wallet_balance()
print(f"Balance: {balance}")

Balance: 962.76372892


In [26]:
fees = api.get_fees('BTCUSDT')
print(f"Taker fees: {fees[0]}")
print(f"Maker fees: {fees[1]}")

Taker fees: 0.00055
Maker fees: 0.0002


### Position info

In [52]:
pos = api.get_positions(symbols)
pos

{'BTCUSDT': {'size': 0.005,
  'aep': 86019.3,
  'mark_price': 85883.2,
  'value': 430.0965,
  'leverage': 100.0,
  'position_balance': 0.0,
  'unrealized_pnl': -0.6805,
  'realized_pnl': -0.23655308},
 'ETHUSDT': {'size': 0,
  'aep': 0,
  'mark_price': 2244.52,
  'value': 0,
  'leverage': 100.0,
  'position_balance': 0,
  'unrealized_pnl': 0,
  'realized_pnl': 0.0}}

### Trading

In [34]:
# Place market order
res = api.place_order('BTCUSDT', 0.005)
res

{'order_id': 'e571d640-49a1-4c32-94d4-e5dd481b701a',
 'ret_code': 0,
 'time': 1740908802469}

In [46]:
balance = api.get_wallet_balance()
print(f"Balance: {balance}")

Balance: 957.91587957


In [36]:
pos = api.get_position(symbols[0])
pos

{'size': 0.005,
 'aep': 86019.3,
 'mark_price': 85961.15,
 'value': 430.0965,
 'leverage': 100.0,
 'position_balance': 0.0,
 'unrealized_pnl': -0.29075,
 'realized_pnl': -0.23655308}

In [54]:
pos = api.get_positions([symbols[0]])
diff = (pos['mark_price']-pos['aep'])*0.005
diff2 = pos['unrealized_pnl']
assert abs(diff2-diff) < 1e-7

In [55]:
resp = api.close_positions(symbols)
resp

{'BTCUSDT': 0, 'ETHUSDT': None}

In [56]:
api.get_positions(symbols)

{'BTCUSDT': {'size': 0,
  'aep': 0,
  'mark_price': 85821.3,
  'value': 0,
  'leverage': 100.0,
  'position_balance': 0,
  'unrealized_pnl': 0,
  'realized_pnl': 0.0},
 'ETHUSDT': {'size': 0,
  'aep': 0,
  'mark_price': 2243.99,
  'value': 0,
  'leverage': 100.0,
  'position_balance': 0,
  'unrealized_pnl': 0,
  'realized_pnl': 0.0}}